In [ ]:
!pip install --quiet jupyterlab-vim jupytex
!jupyter labextension enable

%load_ext autoreload
%autoreload 2

# Langchain

In [3]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j

In [39]:
from langchain.graphs import Neo4jGraph

URI = "bolt://neo4j:7687"
username ="neo4j"
password = "testtest"
AUTH = (username, password)
graph = Neo4jGraph(
    url=URI,
    username=username,
    password=password
)

In [40]:
# Delete everything.
driver = GraphDatabase.driver(URI, auth=AUTH)
driver.execute_query(
    "MATCH(n) OPTIONAL MATCH (n)-[r]-() DELETE n, r"
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff6ed5c820>, keys=[])

In [42]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = URI
os.environ["NEO4J_USERNAME"] = username
os.environ["NEO4J_PASSWORD"] = password

#database = "langchain"
graph = Neo4jGraph()

In [34]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

In [35]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and
chemist who conducted pioneering research on radioactivity. She was the first
woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and
the only person to win a Nobel Prize in two scientific fields. Her husband,
Pierre Curie, was a co-winner of her first Nobel Prize, making them the
first-ever married couple to win the Nobel Prize and launching the Curie family
legacy of five Nobel Prizes. She was, in 1906, the first woman to become a
professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='1867', type='Year'), Node(id='Polish', type='Nationality'), Node(id='Naturalised-French', type='Nationality'), Node(id='Physicist', type='Profession'), Node(id='Chemist', type='Profession'), Node(id='Nobel Prize', type='Award'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='1867', type='Year'), type='BORN_IN'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Naturalised-French', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Physicist', type='Profession'), type='PROFESSION'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Chemist', type='Profession'),

In [45]:
print(graph_documents[0].nodes)

[Node(id='Marie Curie', type='Person'), Node(id='1867', type='Year'), Node(id='Polish', type='Nationality'), Node(id='Naturalised-French', type='Nationality'), Node(id='Physicist', type='Profession'), Node(id='Chemist', type='Profession'), Node(id='Nobel Prize', type='Award'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')]


In [46]:
graph.add_graph_documents(graph_documents)

In [47]:
import hneo4j
from neo4j import GraphDatabase, RoutingControl

driver = GraphDatabase.driver(URI, auth=(username, password))

net = hneo4j.plot_graph(driver)

net.show("graph.html")

graph.html
